<a href="https://colab.research.google.com/github/GitAmitesh/SkinDiseaseClassificationCNN/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Skin Disease Classification using CNN


### Imports

In [ ]:
import zipfile
import os
from google.colab import drive
import pandas as pd
import glob


### Files extraction

In [ ]:
!fusermount -u drive
drive.mount('/content/drive')

# Define the path of your ZIP file
zip_path = "/content/drive/MyDrive/HAM10000/HAM10000.zip"  # Change this to your actual file path
extraction_path = "/content/drive/MyDrive/HAM10000/"  # Change this to your desired extraction path

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)  # Extract into a specific folder

print("Dataset Extracted Successfully!")

In [ ]:
print("Folders and files inside dataset:")
print(os.listdir(dataset_path))  # Lists extracted files & folders

In [ ]:
# Load metadata CSV file
metadata_path = os.path.join(extraction_path, "HAM10000_metadata.csv")  # Ensure correct file name
df = pd.read_csv(metadata_path)

# Display first few rows
df.head()

In [ ]:
# Get all image file paths
image_paths1 = glob.glob(os.path.join(dataset_path, "images_part1", "*.jpg"))
image_paths2 = glob.glob(os.path.join(dataset_path, "images_part2", "*.jpg"))

# Merge both image lists
all_image_paths = image_paths1 + image_paths2

# Convert list to dictionary for easy lookup
image_path_dict = {os.path.basename(path): path for path in all_image_paths}

# Add image paths to DataFrame
df["image_path"] = df["image_id"].map(lambda id: image_path_dict.get(id + ".jpg"))

# Display updated DataFrame
df.head()

In [ ]:
import matplotlib.pyplot as plt
import cv2

# Load and display a sample image
sample_image_path = df.iloc[0]["image_path"]
image = cv2.imread(sample_image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB for proper visualization

plt.imshow(image)
plt.axis("off")
plt.title(f"Sample Image - {df.iloc[0]['dx']}")
plt.show()